In [31]:
import time
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
import chromedriver_binary
import pandas as pd

browser = webdriver.Chrome()
browser.get("https://stock-pictures.netlify.app")

### Auto Infinite Scroll

In [32]:
# Help to scroll the page till we get the bottom
for i in tqdm(range(0,2500000,1000)): # increase the range for infinite scrolling till you reach bottom of the page
    browser.execute_script("window.scrollTo(0,"+ str(i) +")")
    time.sleep(.1)
    # Here, only a few images were extracted

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2500/2500 [04:46<00:00,  8.74it/s]


In [44]:
soup = BeautifulSoup(browser.page_source,'html.parser')

In [46]:
data = []
for sp in tqdm(soup.find_all('div',class_='container')):
    img_link = sp.find('img').get('src')
    tags     = list(set(sp.find('div',class_='tags').text[7:].split(",")))
    likes    = int(sp.find('div',class_='likes-comments').find_all('span')[0].text.strip()[:-6])
    comments = int(sp.find('div',class_='likes-comments').find_all('span')[1].text.strip()[:-9])
    
    data.append([img_link,tags,likes,comments])
    

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 146/146 [00:00<00:00, 9009.52it/s]


In [66]:
# Saving data in DataFrame
df = pd.DataFrame(data,columns=['img_link','tags','likes','comments'])

In [48]:
df.head()

,img_link,tags,likes,comments
0,https://cdn.pixabay.com/photo/2022/03/06/05/30...,"[ Blue Sky, Sky, Atmosphere, Clouds]",196,55
1,https://cdn.pixabay.com/photo/2022/04/07/11/45...,"[ Hummingbird, Ornithology, Bird]",76,20
2,https://cdn.pixabay.com/photo/2022/02/28/15/28...,"[ Rainbow, Rainfall, Subtropical, Sea]",282,106
3,https://cdn.pixabay.com/photo/2022/04/04/02/52...,"[ Japan, Road, Cherry Blossoms, Sakura]",42,11
4,https://cdn.pixabay.com/photo/2022/04/09/18/06...,"[ Flower, Plant, Cape Marguerite]",39,15


In [49]:
df.isna().sum()

img_link    0
tags        0
likes       0
comments    0
dtype: int64

In [50]:
# Checking for duplicate values
df['img_link'].describe()

count                                                   146
unique                                                  145
top       https://cdn.pixabay.com/photo/2022/04/10/19/28...
freq                                                      2
Name: img_link, dtype: object

In [54]:
# Returns no. of unique links
len(list(set(df['img_link'])))

145

In [75]:
df['tags'] = df['tags'].apply(tuple)

In [78]:
# Removing duplicate values
df = df.drop_duplicates()

In [79]:
# Saving the dataframe as csv
df.to_csv('ImagesData.csv',index=False)